# Exercise - Part 1

### Here we will do the cleaning of the data during the scraping operation itself. We will remove the postal codes with no boroughs assigned and also do all the required formattings.

In [17]:
from bs4 import BeautifulSoup
import requests
import csv
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(source, 'lxml')
table = soup.find('table')
tbody = table.find('tbody')

csv_file = open('canada_ps.csv', 'w')
csv_writer = csv.writer(csv_file)
csv_writer.writerow(['PostalCode', 'Borough', 'Neighborhood'])

for tr in tbody.find_all('tr')[1:]:
    post = tr.find_all('td')[0].text.strip()
    borough = tr.find_all('td')[1].text.strip()
    neighborhood = tr.find_all('td')[2].text.strip()
    if borough != 'Not assigned':
        if (len(neighborhood.strip()) == 0):
            neighborhood = borough
        try:
            neighborhood = neighborhood.replace(" /",",")
        except:
            pass
        csv_writer.writerow([post, borough, neighborhood])
        
csv_file.close()

Now, we have the formatted data stored in the csv file. Let us now import the data as a pandas data frame

In [49]:
import pandas as pd
df = pd.read_csv('canada_ps.csv')

In [50]:
df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [51]:
df.shape

(103, 3)

Now, let us add the latitudes and longitudes using the geocorder package

Tried to use the geocorder package, but it is taking a lot of time to fetch the locations. So let us use the provided link to get the location data

In [43]:
!wget -O canada_loc.csv http://cocl.us/Geospatial_data

--2020-04-24 12:41:29--  http://cocl.us/Geospatial_data
Resolving cocl.us (cocl.us)... 159.8.69.21, 159.8.72.228, 159.8.69.24
Connecting to cocl.us (cocl.us)|159.8.69.21|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://cocl.us/Geospatial_data [following]
--2020-04-24 12:41:29--  https://cocl.us/Geospatial_data
Connecting to cocl.us (cocl.us)|159.8.69.21|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2020-04-24 12:41:31--  https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv
Resolving ibm.box.com (ibm.box.com)... 107.152.26.197
Connecting to ibm.box.com (ibm.box.com)|107.152.26.197|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2020-04-24 12:41:32--  https://ibm.box.com/public/static/9a

In [44]:
df_loc = pd.read_csv('canada_loc.csv')

In [45]:
df_loc.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [54]:
df_loc.rename(columns = {'Postal Code':'PostalCode'}, inplace = True)
df_merged = df.merge(df_loc, on='PostalCode', how='left')
df_merged.head(10)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
